# ExoNAMD API v1.0

## Summary

The observer wants to compute the relative and/or absolute NAMD of a given planetary system, a subset of planetary systems, or all the known systems.

This tool handles all of the above.

### Task 1: getting the data

This task retrieves the system parameters from the NASA Exoplanet Archive database, and stores them in a local database.